# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

In [14]:
EPHYS_SAMPLE_RATE = 20000

Explanation of each input and where it comes from.

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_long_comp"

In [16]:
# TRIAL_LABELS_DF = pd.read_excel("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/data/rce_pilot_3_long_comp_per_video_trial_labels.xlsx")
TRIAL_LABELS_DF = pd.read_excel("../../data/{}_per_video_trial_labels.xlsx".format(OUTPUT_PREFIX))

In [17]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=[col for col in TRIAL_LABELS_DF if "unnamed" in col.lower()], errors="ignore")

In [18]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.rename(columns={col: col.strip() for col in TRIAL_LABELS_DF}, errors="ignore")

In [19]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_03_spectral_bands.pkl".format(OUTPUT_PREFIX))

In [20]:
# LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_04_spectral_and_sleap.pkl".format(OUTPUT_PREFIX))

In [21]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.drop(columns=[col for col in LFP_SPECTRAL_DF if "unnamed" in col.lower()], errors="ignore")

In [22]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.rename(columns={col: col.strip() for col in LFP_SPECTRAL_DF}, errors="ignore")

In [23]:
cluster_range_columns = pd.read_pickle("../2024_06_26_sleap_clustering/proc/cluster_range_columns.pkl")

In [24]:
FULL_LFP_TRACES_PKL = "{}_10_per_trial_spectral_bans_sleap.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [25]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [26]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'notes', 'session_path',
       'recording', 'current_subject',
       ...
       'vHPC_power_theta', 'vHPC_power_gamma', 'BLA_power_theta',
       'BLA_power_gamma', 'LH_power_theta', 'LH_power_gamma', 'MD_power_theta',
       'MD_power_gamma', 'mPFC_power_theta', 'mPFC_power_gamma'],
      dtype='object', length=155)

In [27]:
TRIAL_LABELS_DF.columns

Index(['session_dir', 'tone_start_frame', 'reward_start', 'reward_dispensed',
       'tone_stop_frame', 'condition', 'competition_closeness',
       'get_reward_frame', 'out_reward_frame', 'notes', 'experiment',
       'box_1_port_entry_frames', 'box_2_port_entry_frames', 'video_name',
       'all_subjects', 'first_timestamp', 'last_timestamp',
       'tone_start_timestamp', 'tone_stop_timestamp',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps'],
      dtype='object')

## Merging the labels and LFP spectral

In [28]:
TRIAL_LABELS_DF.head()

,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,...,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240317_151922_long_comp_subj_3-1_and_3-3,1,81,101,88,NaN,NaN,NaN,NaN,NaN,...,[ 1 88],NaN,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"['3.1', '3.3']",2415571,48236575,0,87540,[ 0 87540],NaN
1,20240317_151922_long_comp_subj_3-1_and_3-3,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,NaN,...,[1300 1315] [1371 1380] [1420 1513],NaN,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"['3.1', '3.3']",2415571,48236575,1287955,1487958,[1301956 1317753] [1373156 1382756] [1422554 1...,NaN
2,20240317_151922_long_comp_subj_3-1_and_3-3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,NaN,...,[3277 3298] [3310 3356] [3357 3368] [3377 3378...,NaN,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"['3.1', '3.3']",2415571,48236575,3287980,3487980,[3281780 3302780] [3315978 3361781] [3362381 3...,NaN
3,20240317_151922_long_comp_subj_3-1_and_3-3,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,"3.3 walk towards the reward port, and then wal...",...,[4383 4502] [4527 4851],NaN,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"['3.1', '3.3']",2415571,48236575,4487995,4687995,[4390791 4509995] [4535398 4859200],NaN
4,20240317_151922_long_comp_subj_3-1_and_3-3,5478,5558,5578,5678,3.1,Subj 1 blocking Subj 2,NaN,NaN,3.1 blocked 3.3 from approaching at frame 5574,...,[5588 5778],NaN,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"['3.1', '3.3']",2415571,48236575,5488007,5688010,[5597806 5788609],NaN


In [29]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,vHPC_power_theta,vHPC_power_gamma,BLA_power_theta,BLA_power_gamma,LH_power_theta,LH_power_gamma,MD_power_theta,MD_power_gamma,mPFC_power_theta,mPFC_power_gamma
0,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[1, 88], [1286, 1486], [3282, 3482], [4480, 4...","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-1_t6b6_merged,3.1,...,"[0.029989537, 0.05074093, 0.045202438, 0.03654...","[0.0021393485, 0.001990721, 0.0016591972, 0.00...","[0.01576374, 0.017160619, nan, nan, 0.01513487...","[0.0029858148, 0.0055226046, nan, nan, 0.00234...","[0.014374395, nan, nan, nan, nan, nan, 0.02792...","[0.0015033478, nan, nan, nan, nan, nan, 0.0011...","[0.015749235, nan, nan, 0.013588659, 0.0394984...","[0.0012602397, nan, nan, 0.0016373609, 0.00110...","[0.02549853, 0.01599767, 0.019950056, 0.041470...","[0.0018852353, 0.0019809369, 0.0013036462, 0.0..."
1,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[2, 132], [1922, 2220], [4906, 5203], [6696, ...","[[2, 132], [387, 415], [463, 546], [548, 571],...",[],20240317_151922_long_comp_subj_3-1_and_3-3.2.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-1_t6b6_merged,3.1,...,"[0.029989537, 0.05074093, 0.045202438, 0.03654...","[0.0021393485, 0.001990721, 0.0016591972, 0.00...","[0.01576374, 0.017160619, nan, nan, 0.01513487...","[0.0029858148, 0.0055226046, nan, nan, 0.00234...","[0.014374395, nan, nan, nan, nan, nan, 0.02792...","[0.0015033478, nan, nan, nan, nan, nan, 0.0011...","[0.015749235, nan, nan, 0.013588659, 0.0394984...","[0.0012602397, nan, nan, 0.0016373609, 0.00110...","[0.02549853, 0.01599767, 0.019950056, 0.041470...","[0.0018852353, 0.0019809369, 0.0013036462, 0.0..."
2,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[1, 88], [1286, 1486], [3282, 3482], [4480, 4...","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-3_t5b5_merged,3.3,...,"[0.030139383, 0.038337562, 0.035997633, 0.0163...","[0.0013218134, 0.001314961, 0.0014802001, 0.00...","[0.035382226, 0.031998232, 0.029764285, 0.0189...","[0.0013072899, 0.0012992765, 0.0010765253, 0.0...","[0.03857511, 0.028973801, 0.029779362, 0.02279...","[0.0009867165, 0.0008933175, 0.00090869045, 0....","[0.01698081, 0.014525875, 0.015493691, 0.01093...","[0.0012375844, 0.0010923791, 0.0008535082, 0.0...","[0.020940535, 0.027591836, 0.029557036, 0.0211...","[0.0014317904, 0.0010592391, 0.0012028972, 0.0..."
3,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[2, 132], [1922, 2220], [4906, 5203], [6696, ...","[[2, 132], [387, 415], [463, 546], [548, 571],...",[],20240317_151922_long_comp_subj_3-1_and_3-3.2.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-3_t5b5_merged,3.3,...,"[0.030139383, 0.038337562, 0.035997633, 0.0163...","[0.0013218134, 0.001314961, 0.0014802001, 0.00...","[0.035382226, 0.031998232, 0.029764285, 0.0189...","[0.0013072899, 0.0012992765, 0.0010765253, 0.0...","[0.03857511, 0.028973801, 0.029779362, 0.02279...","[0.0009867165, 0.0008933175, 0.00090869045, 0....","[0.01698081, 0.014525875, 0.015493691, 0.01093...","[0.0012375844, 0.0010923791, 0.0008535082, 0.0...","[0.020940535, 0.027591836, 0.029557036, 0.0211...","[0.0014317904, 0.0010592391, 0.0012028972, 0.0..."
4,rce_pilot_3,20240317_172017_long_comp_subj_4-2_and_4-3,"[[1, 102], [1300, 1501], [3297, 3497], [4494, ...","[[1, 102], [115, 153], [191, 245], [261, 263],...",[],20240317_172017_long_comp_subj_4-2_and_4-3.1.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_172017_long_comp_subj_4-2_t6b6_merged,4.2,...,"[0.0

# Dropping redundant columns

In [30]:
dropping_columns = list(set(TRIAL_LABELS_DF.columns).intersection(set(LFP_SPECTRAL_DF.columns)))

In [31]:
dropping_columns

['box_1_port_entry_timestamps',
 'first_timestamp',
 'box_2_port_entry_frames',
 'all_subjects',
 'video_name',
 'last_timestamp',
 'box_2_port_entry_timestamps',
 'session_dir',
 'box_1_port_entry_frames',
 'notes']

In [32]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=[col for col in dropping_columns if col != "video_name"], errors="ignore")

## Merging the LFP and trial dataframe

In [33]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [34]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20240317_151922_long_comp_subj_3-1_and_3-3.1',
       '20240317_151922_long_comp_subj_3-1_and_3-3.2',
       '20240317_172017_long_comp_subj_4-2_and_4-3.1',
       '20240317_172017_long_comp_subj_4-2_and_4-3.2',
       '20240318_143819_long_comp_subj_3-3_and_3-4.1',
       '20240318_170933_long_comp_subj_4-3_and_4-4.1',
       '20240319_134914_long_comp_subj_3-1_and_3-4.1',
       '20240319_160457_long_comp_subj_4-2_and_4-4.1',
       '20240320_114629_long_comp_subj_5-3_and_5-4.1',
       '20240321_114851_long_comp_subj_5-2_and_5-3.1'], dtype=object)

In [35]:
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str)


In [36]:
LFP_SPECTRAL_DF["current_subject"].head()

0    3.1
1    3.1
2    3.3
3    3.3
4    4.2
Name: current_subject, dtype: object

In [37]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["video_name"])

In [38]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition"])

In [39]:
TRIAL_LABELS_DF["video_name"] = TRIAL_LABELS_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [40]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20240317_151922_long_comp_subj_3-1_and_3-3.1',
       '20240317_172017_long_comp_subj_4-2_and_4-3.1',
       '20240318_143819_long_comp_subj_3-3_and_3-4.1',
       '20240318_170933_long_comp_subj_4-3_and_4-4.1',
       '20240319_134914_long_comp_subj_3-1_and_3-4.1',
       '20240319_160457_long_comp_subj_4-2_and_4-4.1',
       '20240320_114629_long_comp_subj_5-3_and_5-4.1',
       '20240321_114851_long_comp_subj_5-2_and_5-3.1'], dtype=object)

In [41]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["video_name"], how="inner", suffixes=('', '_remove'))

In [42]:
TRIALS_AND_SPECTRAL_DF[[col for col in TRIALS_AND_SPECTRAL_DF if "_remove" in col]].head()

""
0
1
2
3
4


In [43]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "remove" in col], errors="ignore")

In [44]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "Unnamed" in col])

In [45]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,vHPC_power_theta,vHPC_power_gamma,BLA_power_theta,BLA_power_gamma,LH_power_theta,LH_power_gamma,MD_power_theta,MD_power_gamma,mPFC_power_theta,mPFC_power_gamma
0,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.029989537, 0.05074093, 0.045202438, 0.03654...","[0.0021393485, 0.001990721, 0.0016591972, 0.00...","[0.01576374, 0.017160619, nan, nan, 0.01513487...","[0.0029858148, 0.0055226046, nan, nan, 0.00234...","[0.014374395, nan, nan, nan, nan, nan, 0.02792...","[0.0015033478, nan, nan, nan, nan, nan, 0.0011...","[0.015749235, nan, nan, 0.013588659, 0.0394984...","[0.0012602397, nan, nan, 0.0016373609, 0.00110...","[0.02549853, 0.01599767, 0.019950056, 0.041470...","[0.0018852353, 0.0019809369, 0.0013036462, 0.0..."
1,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.030139383, 0.038337562, 0.035997633, 0.0163...","[0.0013218134, 0.001314961, 0.0014802001, 0.00...","[0.035382226, 0.031998232, 0.029764285, 0.0189...","[0.0013072899, 0.0012992765, 0.0010765253, 0.0...","[0.03857511, 0.028973801, 0.029779362, 0.02279...","[0.0009867165, 0.0008933175, 0.00090869045, 0....","[0.01698081, 0.014525875, 0.015493691, 0.01093...","[0.0012375844, 0.0010923791, 0.0008535082, 0.0...","[0.020940535, 0.027591836, 0.029557036, 0.0211...","[0.0014317904, 0.0010592391, 0.0012028972, 0.0..."
2,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.029989537, 0.05074093, 0.045202438, 0.03654...","[0.0021393485, 0.001990721, 0.0016591972, 0.00...","[0.01576374, 0.017160619, nan, nan, 0.01513487...","[0.0029858148, 0.0055226046, nan, nan, 0.00234...","[0.014374395, nan, nan, nan, nan, nan, 0.02792...","[0.0015033478, nan, nan, nan, nan, nan, 0.0011...","[0.015749235, nan, nan, 0.013588659, 0.0394984...","[0.0012602397, nan, nan, 0.0016373609, 0.00110...","[0.02549853, 0.01599767, 0.019950056, 0.041470...","[0.0018852353, 0.0019809369, 0.0013036462, 0.0..."
3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.030139383, 0.038337562, 0.035997633, 0.0163...","[0.0013218134, 0.001314961, 0.0014802001, 0.00...","[0.035382226, 0.031998232, 0.029764285, 0.0189...","[0.0013072899, 0.0012992765, 0.0010765253, 0.0...","[0.03857511, 0.028973801, 0.029779362, 0.02279...","[0.0009867165, 0.0008933175, 0.00090869045, 0....","[0.01698081, 0.014525875, 0.015493691, 0.01093...","[0.0012375844, 0.0010923791, 0.0008535082, 0.0...","[0.020940535, 0.027591836, 0.029557036, 0.0211...","[0.0014317904, 0.0010592391, 0.0012028972, 0.0..."
4,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.029989537, 0.05074093, 0.045202438, 0.03654...","[0.0021393485, 0.001990721, 0.0016591972, 0.00...","[0.01576374, 0.017160619, nan, nan, 0.01513487...","[0.0029858148, 0.0055226046, nan, nan, 0.00234...","[0.014374395, nan, nan, nan, nan, nan, 0.02792...","[0.0015033478, nan, nan, nan, nan, nan, 0.0011...","[0.015749235, nan, nan, 0.013588659, 0.0394984...","[0.0012602397, nan, nan, 0.0016373609, 0.00110...","[0.02549853, 0.01599767, 0.019950056, 0.041470...","[0.0018852353, 0.0019809369, 0.0013036462, 0.0..."


In [46]:
TRIALS_AND_SPECTRAL_DF.tail()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,vHPC_power_theta,vHPC_power_gamma,BLA_power_theta,BLA_power_gamma,LH_power_theta,LH_power_gamma,MD_power_theta,MD_power_gamma,mPFC_power_theta,mPFC_power_gamma
629,51909,51989,52009,52108,5.3,Subj 2 then Subj 1,51972.0,52121.0,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,"[0.013462405, 0.01815342, 0.03453803, 0.042738...","[0.0016632699, 0.0021446128, 0.0014671937, 0.0...","[0.03483833, 0.037853822, 0.036286745, 0.04037...","[0.0015041223, 0.0011649611, 0.0013518208, 0.0...","[0.009690219, 0.015778568, 0.022578647, 0.0171...","[0.0003425703, 0.00045422354, 0.0006697395, 0....","[0.04274923, 0.04190998, 0.05191328, 0.0686178...","[0.0005139264, 0.00046058508, 0.0005998925, 0....","[0.0064892564, 0.015247796, 0.028501984, 0.033...","[0.0009644553, 0.0007621409, 0.0011118953, 0.0..."
630,54104,54184,54204,54304,5.3,Subj 2 then Subj 1,54157.0,54286.0,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,"[0.011295435, 0.010506853, 0.014838293, 0.0172...","[0.0003503056, 0.00036143616, 0.00019823617, 0...","[0.015539557, 0.013323676, 0.018323993, 0.0222...","[0.00048828806, 0.00052347546, 0.00027870925, ...","[nan, nan, nan, nan, nan, 0.04063613, 0.025161...","[nan, nan, nan, nan, nan, 0.0008793536, 0.0008...","[nan, nan, 0.0924906, nan, nan, 0.04832769, 0....","[nan, nan, 0.0011266877, nan, nan, 0.000791560...","[0.040000618, 0.026731346, 0.03864481, nan, na...","[0.0015003057, 0.0017328794, 0.0015343259, nan..."
631,54104,54184,54204,54304,5.3,Subj 2 then Subj 1,54157.0,54286.0,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,"[0.013462405, 0.01815342, 0.03453803, 0.042738...","[0.0016632699, 0.0021446128, 0.0014671937, 0.0...","[0.03483833, 0.037853822, 0.036286745, 0.04037...","[0.0015041223, 0.0011649611, 0.0013518208, 0.0...","[0.009690219, 0.015778568, 0.022578647, 0.0171...","[0.0003425703, 0.00045422354, 0.0006697395, 0....","[0.04274923, 0.04190998, 0.05191328, 0.0686178...","[0.0005139264, 0.00046058508, 0.0005998925, 0....","[0.0064892564, 0.015247796, 0.028501984, 0.033...","[0.0009644553, 0.0007621409, 0.0011118953, 0.0..."
632,55203,55283,55303,55402,tie,Close Call,NaN,NaN,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,"[0.011295435, 0.010506853, 0.014838293, 0.0172...","[0.0003503056, 0.00036143616, 0.00019823617, 0...","[0.015539557, 0.013323676, 0.018323993, 0.0222...","[0.00048828806, 0.00052347546, 0.00027870925, ...","[nan, nan, nan, nan, nan, 0.04063613, 0.025161...","[nan, nan, nan, nan, nan, 0.0008793536, 0.0008...","[nan, nan, 0.0924906, nan, nan, 0.04832769, 0....","[nan, nan, 0.0011266877, nan, nan, 0.000791560...","[0.040000618, 0.026731346, 0.03864481, nan, na...","[0.0015003057, 0.0017328794, 0.0015343259, nan..."
633,55203,55283,55303,55402,tie,Close Call,NaN,NaN,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,"[0.013462405, 0.01815342, 0.03453803, 0.042738...","[0.0016632699, 0.0021446128, 0.0014671937, 0.0...","[0.03483833, 0.037853822, 0.036286745, 0.04037...","[0.0015041223, 0.0011649611, 0.0013518208, 0.0...","[0.009690219, 0.015778568, 0.022578647, 0.0171...","[0.0003425703, 0.00045422354, 0.0006697395, 0....","[0.04274923, 0.04190998, 0.05191328, 0.0686178...","[0.0005139264, 0.00046058508, 0.0005998925, 0....","[0.0064892564, 0.015247796, 0.028501984, 0.033...","[0.0009644553, 0.0007621409, 0.0011118953, 0.0..."


In [47]:
timestamps_col = [col for col in TRIALS_AND_SPECTRAL_DF if "timestamp" in col and "timestamps" not in col and "cluster" not in col]

In [48]:
for col in timestamps_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF[col].astype(int)

In [49]:
TRIALS_AND_SPECTRAL_DF[timestamps_col].head()

,tone_start_timestamp,tone_stop_timestamp,first_timestamp,last_timestamp
0,1287955,1487958,2415571,48236575
1,1287955,1487958,2415571,48236575
2,3287980,3487980,2415571,48236575
3,3287980,3487980,2415571,48236575
4,4487995,4687995,2415571,48236575


In [50]:
frame_col = [col for col in TRIALS_AND_SPECTRAL_DF if "frame" in col and "tone" in col and "frames" not in col]

In [51]:
for col in frame_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF[col].astype(int)

In [52]:
TRIALS_AND_SPECTRAL_DF[frame_col].head()

,tone_start_frame,tone_stop_frame
0,1286,1486
1,1286,1486
2,3282,3482
3,3282,3482
4,4480,4679


## Adding the SLEAP data

In [53]:
cluster_range_columns.columns

Index(['video_name', 'current_subject', 'tone_start_frame', 'agent_locations',
       'agent_nose', 'agent_nose_to_reward_port', 'agent_tail_base',
       'agent_thorax', 'agent_thorax_velocity', 'agent_to_reward_port_angle',
       'closebool_agent_nose_to_reward_port',
       'closebool_subject_nose_to_reward_port', 'clusterable_embedding_x',
       'clusterable_embedding_y', 'competitiveness', 'frame_index',
       'kmeans_cluster', 'manual_cluster_id',
       'movingbool_agent_thorax_velocity',
       'movingbool_subject_thorax_velocity', 'nose_to_reward_port_diff',
       'nose_to_reward_port_sum', 'nose_to_tail_diff', 'nose_to_tail_sum',
       'standard_embedding_x', 'standard_embedding_y', 'subject_locations',
       'subject_nose', 'subject_nose_to_reward_port', 'subject_tail_base',
       'subject_thorax', 'subject_thorax_to_agent_thorax',
       'subject_thorax_velocity', 'subject_to_reward_port_angle',
       'thorax_velocity_diff', 'thorax_velocity_sum',
       'to_reward_

In [54]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,vHPC_power_theta,vHPC_power_gamma,BLA_power_theta,BLA_power_gamma,LH_power_theta,LH_power_gamma,MD_power_theta,MD_power_gamma,mPFC_power_theta,mPFC_power_gamma
0,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.029989537, 0.05074093, 0.045202438, 0.03654...","[0.0021393485, 0.001990721, 0.0016591972, 0.00...","[0.01576374, 0.017160619, nan, nan, 0.01513487...","[0.0029858148, 0.0055226046, nan, nan, 0.00234...","[0.014374395, nan, nan, nan, nan, nan, 0.02792...","[0.0015033478, nan, nan, nan, nan, nan, 0.0011...","[0.015749235, nan, nan, 0.013588659, 0.0394984...","[0.0012602397, nan, nan, 0.0016373609, 0.00110...","[0.02549853, 0.01599767, 0.019950056, 0.041470...","[0.0018852353, 0.0019809369, 0.0013036462, 0.0..."
1,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.030139383, 0.038337562, 0.035997633, 0.0163...","[0.0013218134, 0.001314961, 0.0014802001, 0.00...","[0.035382226, 0.031998232, 0.029764285, 0.0189...","[0.0013072899, 0.0012992765, 0.0010765253, 0.0...","[0.03857511, 0.028973801, 0.029779362, 0.02279...","[0.0009867165, 0.0008933175, 0.00090869045, 0....","[0.01698081, 0.014525875, 0.015493691, 0.01093...","[0.0012375844, 0.0010923791, 0.0008535082, 0.0...","[0.020940535, 0.027591836, 0.029557036, 0.0211...","[0.0014317904, 0.0010592391, 0.0012028972, 0.0..."
2,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.029989537, 0.05074093, 0.045202438, 0.03654...","[0.0021393485, 0.001990721, 0.0016591972, 0.00...","[0.01576374, 0.017160619, nan, nan, 0.01513487...","[0.0029858148, 0.0055226046, nan, nan, 0.00234...","[0.014374395, nan, nan, nan, nan, nan, 0.02792...","[0.0015033478, nan, nan, nan, nan, nan, 0.0011...","[0.015749235, nan, nan, 0.013588659, 0.0394984...","[0.0012602397, nan, nan, 0.0016373609, 0.00110...","[0.02549853, 0.01599767, 0.019950056, 0.041470...","[0.0018852353, 0.0019809369, 0.0013036462, 0.0..."
3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.030139383, 0.038337562, 0.035997633, 0.0163...","[0.0013218134, 0.001314961, 0.0014802001, 0.00...","[0.035382226, 0.031998232, 0.029764285, 0.0189...","[0.0013072899, 0.0012992765, 0.0010765253, 0.0...","[0.03857511, 0.028973801, 0.029779362, 0.02279...","[0.0009867165, 0.0008933175, 0.00090869045, 0....","[0.01698081, 0.014525875, 0.015493691, 0.01093...","[0.0012375844, 0.0010923791, 0.0008535082, 0.0...","[0.020940535, 0.027591836, 0.029557036, 0.0211...","[0.0014317904, 0.0010592391, 0.0012028972, 0.0..."
4,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.029989537, 0.05074093, 0.045202438, 0.03654...","[0.0021393485, 0.001990721, 0.0016591972, 0.00...","[0.01576374, 0.017160619, nan, nan, 0.01513487...","[0.0029858148, 0.0055226046, nan, nan, 0.00234...","[0.014374395, nan, nan, nan, nan, nan, 0.02792...","[0.0015033478, nan, nan, nan, nan, nan, 0.0011...","[0.015749235, nan, nan, 0.013588659, 0.0394984...","[0.0012602397, nan, nan, 0.0016373609, 0.00110...","[0.02549853, 0.01599767, 0.019950056, 0.041470...","[0.0018852353, 0.0019809369, 0.0013036462, 0.0..."


In [55]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1287955
1    1287955
2    3287980
3    3287980
4    4487995
Name: tone_start_timestamp, dtype: int64

In [56]:
cluster_range_columns["tone_start_timestamp"] = cluster_range_columns["tone_start_timestamp"].astype(int)
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].astype(int)

In [57]:
cluster_range_columns["current_subject"] = cluster_range_columns["current_subject"].apply(lambda x: str(x).strip())
TRIALS_AND_SPECTRAL_DF["current_subject"] = TRIALS_AND_SPECTRAL_DF["current_subject"].apply(lambda x: str(x).strip())

In [58]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIALS_AND_SPECTRAL_DF, cluster_range_columns, on=["video_name", "current_subject", "tone_start_timestamp"], how="left", suffixes=('', '_remove'))

In [59]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"]

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,stop_frame,tone_start_to_stop_frame,tone_stop_frame_remove,tone_stop_timestamp_remove,tracked_subject,video_id,cluster_index_ranges_dict,competitiveness_index_ranges_dict,cluster_frame_ranges_dict,competitiveness_frame_ranges_dict
0,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,45736.0,"(1286, 1486)",1486.0,1487958.0,"[3.1, 3.3]",0.0,"{'4': [(0, 47), (113, 138), (303, 346), (363, ...","{'no_comp': [(0, 75), (96, 169), (213, 225), (...","{'4': [(1086, 1133), (1199, 1224), (1389, 1432...","{'no_comp': [(1086, 1161), (1182, 1255), (1299..."
1,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,45736.0,"(1286, 1486)",1486.0,1487958.0,"[3.1, 3.3]",0.0,"{'4': [(0, 47), (113, 138), (303, 346), (363, ...","{'no_comp': [(0, 75), (96, 169), (213, 225), (...","{'4': [(1086, 1133), (1199, 1224), (1389, 1432...","{'no_comp': [(1086, 1161), (1182, 1255), (1299..."
2,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,45736.0,"(3282, 3482)",3482.0,3487980.0,"[3.1, 3.3]",0.0,"{'1': [(0, 3), (63, 82), (105, 115), (174, 184...","{'no_comp': [(0, 184), (188, 476), (497, 527),...","{'1': [(3082, 3085), (3145, 3164), (3187, 3197...","{'no_comp': [(3082, 3266), (3270, 3558), (3579..."
3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,45736.0,"(3282, 3482)",3482.0,3487980.0,"[3.1, 3.3]",0.0,"{'1': [(0, 3), (63, 82), (105, 115), (174, 184...","{'no_comp': [(0, 184), (188, 476), (497, 527),...","{'1': [(3082, 3085), (3145, 3164), (3187, 3197...","{'no_comp': [(3082, 3266), (3270, 3558), (3579..."
4,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,45736.0,"(4480, 4679)",4679.0,4687995.0,"[3.1, 3.3]",0.0,"{'1': [(0, 10), (29, 82), (91, 104), (135, 144...","{'no_comp': [(0, 10), (29, 82), (91, 104), (12...","{'1': [(4280, 4290), (4309, 4362), (4371, 4384...","{'no_comp': [(4280, 4290), (4309, 4362), (4371..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,51909,51989,52009,52108,5.3,Subj 2 then Subj 1,51972.0,52121.0,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,61326.0,"(51909, 52108)",52108.0,52206280.0,"[5.2, 5.3]",6.0,"{'4': [(0, 12), (67, 69), (73, 75)], '1': [(13...","{'no_comp': [(0, 113), (511, 541), (591, 598)]...","{'4': [(51709, 51721), (51776, 51778), (51782,...","{'no_comp': [(51709, 51822), (52220, 52250), (..."
630,54104,54184,54204,54304,5.3,Subj 2 then Subj 1,54157.0,54286.0,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,61326.0,"(54104, 54304)",54304.0,54406307.0,"[5.2, 5.3]",6.0,"{'5': [(0, 5), (17, 30), (90, 108), (354, 368)...","{'competitive': [(0, 44), (76, 119), (160, 206...","{'5': [(53904, 53909), (53921, 53934), (53994,...","{'competitive': [(53904, 53948), (53980, 54023..."
631,54104,54184,54204,54304,5.3,Subj 2 then Subj 1,54157.0,54286.0,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,61326.0,"(54104, 54304)",54304.0,54406307.0,"[5.2, 5.3]",6.0,"{'5': [(0, 5), (17, 30), (90, 108), (354, 368)...","{'competitive': [(0, 44), (76, 119), (160, 206...","{'5': [(53904, 53909), (53921, 53934), (53994,...","{'competitive': [(53904, 53948), (53980, 54023..."
632,55203,55283,55303,55402,tie,Close Call,NaN,NaN,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,61326.0,"(55203, 55402)",55402.0,55506320.0,"[5.2, 5.3]",6.0,"{'7': [(0, 5), (27, 27), (54, 92), (114, 196),...","{'competitive': [(0, 208), (219, 598)], 'no_co...","{'7': [(55003, 55008), (55030, 55030), (55057,...","{'competitive': [(55003, 55211), (55222, 55601..."


# Calculating the relevant timestamps

In [60]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 10 * EPHYS_SAMPLE_RATE

In [61]:
TRIALS_AND_SPECTRAL_DF["post_trial_end_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"] + 10 * EPHYS_SAMPLE_RATE

In [62]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1287955
1    1287955
2    3287980
3    3287980
4    4487995
Name: tone_start_timestamp, dtype: int64

In [63]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1287955
1    1287955
2    3287980
3    3287980
4    4487995
Name: tone_start_timestamp, dtype: int64

In [64]:
TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"].head()

0    1487958
1    1487958
2    3487980
3    3487980
4    4687995
Name: tone_stop_timestamp, dtype: int64

In [65]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,tone_stop_frame_remove,tone_stop_timestamp_remove,tracked_subject,video_id,cluster_index_ranges_dict,competitiveness_index_ranges_dict,cluster_frame_ranges_dict,competitiveness_frame_ranges_dict,baseline_start_timestamp,post_trial_end_timestamp
0,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,1486.0,1487958.0,"[3.1, 3.3]",0.0,"{'4': [(0, 47), (113, 138), (303, 346), (363, ...","{'no_comp': [(0, 75), (96, 169), (213, 225), (...","{'4': [(1086, 1133), (1199, 1224), (1389, 1432...","{'no_comp': [(1086, 1161), (1182, 1255), (1299...",1087955,1687958
1,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,1486.0,1487958.0,"[3.1, 3.3]",0.0,"{'4': [(0, 47), (113, 138), (303, 346), (363, ...","{'no_comp': [(0, 75), (96, 169), (213, 225), (...","{'4': [(1086, 1133), (1199, 1224), (1389, 1432...","{'no_comp': [(1086, 1161), (1182, 1255), (1299...",1087955,1687958
2,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,3482.0,3487980.0,"[3.1, 3.3]",0.0,"{'1': [(0, 3), (63, 82), (105, 115), (174, 184...","{'no_comp': [(0, 184), (188, 476), (497, 527),...","{'1': [(3082, 3085), (3145, 3164), (3187, 3197...","{'no_comp': [(3082, 3266), (3270, 3558), (3579...",3087980,3687980
3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,3482.0,3487980.0,"[3.1, 3.3]",0.0,"{'1': [(0, 3), (63, 82), (105, 115), (174, 184...","{'no_comp': [(0, 184), (188, 476), (497, 527),...","{'1': [(3082, 3085), (3145, 3164), (3187, 3197...","{'no_comp': [(3082, 3266), (3270, 3558), (3579...",3087980,3687980
4,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,4679.0,4687995.0,"[3.1, 3.3]",0.0,"{'1': [(0, 10), (29, 82), (91, 104), (135, 144...","{'no_comp': [(0, 10), (29, 82), (91, 104), (12...","{'1': [(4280, 4290), (4309, 4362), (4371, 4384...","{'no_comp': [(4280, 4290), (4309, 4362), (4371...",4287995,4887995


# Filtering out the SLEAP posed for during trials

In [66]:
TRIALS_AND_SPECTRAL_DF["video_frame"] = TRIALS_AND_SPECTRAL_DF["video_timestamps"].apply(lambda x: np.array(list(range(len(x)))) + 1)

In [67]:
TRIALS_AND_SPECTRAL_DF["video_frame"].head().apply(lambda x: x.shape)

0    (45736,)
1    (45736,)
2    (45736,)
3    (45736,)
4    (45736,)
Name: video_frame, dtype: object

In [68]:
cluster_range_columns.columns

Index(['video_name', 'current_subject', 'tone_start_frame', 'agent_locations',
       'agent_nose', 'agent_nose_to_reward_port', 'agent_tail_base',
       'agent_thorax', 'agent_thorax_velocity', 'agent_to_reward_port_angle',
       'closebool_agent_nose_to_reward_port',
       'closebool_subject_nose_to_reward_port', 'clusterable_embedding_x',
       'clusterable_embedding_y', 'competitiveness', 'frame_index',
       'kmeans_cluster', 'manual_cluster_id',
       'movingbool_agent_thorax_velocity',
       'movingbool_subject_thorax_velocity', 'nose_to_reward_port_diff',
       'nose_to_reward_port_sum', 'nose_to_tail_diff', 'nose_to_tail_sum',
       'standard_embedding_x', 'standard_embedding_y', 'subject_locations',
       'subject_nose', 'subject_nose_to_reward_port', 'subject_tail_base',
       'subject_thorax', 'subject_thorax_to_agent_thorax',
       'subject_thorax_velocity', 'subject_to_reward_port_angle',
       'thorax_velocity_diff', 'thorax_velocity_sum',
       'to_reward_

In [69]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if ("thorax" in col or "nose" in col or "standard_embedding" in col or "cluster" in col or "frame_index" in col or "locations" in col or "tailbase" in col or "competitiveness" in col) and "timestamp" not in col and "dict" not in col]


In [70]:
sleap_columns

['agent_locations',
 'agent_nose',
 'agent_nose_to_reward_port',
 'agent_thorax',
 'agent_thorax_velocity',
 'closebool_agent_nose_to_reward_port',
 'closebool_subject_nose_to_reward_port',
 'clusterable_embedding_x',
 'clusterable_embedding_y',
 'competitiveness',
 'frame_index',
 'kmeans_cluster',
 'manual_cluster_id',
 'movingbool_agent_thorax_velocity',
 'movingbool_subject_thorax_velocity',
 'nose_to_reward_port_diff',
 'nose_to_reward_port_sum',
 'nose_to_tail_diff',
 'nose_to_tail_sum',
 'standard_embedding_x',
 'standard_embedding_y',
 'subject_locations',
 'subject_nose',
 'subject_nose_to_reward_port',
 'subject_thorax',
 'subject_thorax_to_agent_thorax',
 'subject_thorax_velocity',
 'thorax_velocity_diff',
 'thorax_velocity_sum',
 'within_trial_frame_index']

In [71]:
for col in sleap_columns:
    print(col)
    print(TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"][col].apply(lambda x: x.shape if isinstance(x, np.ndarray) else np.nan).head())

agent_locations
0    (600, 6, 2)
1    (600, 6, 2)
2    (600, 6, 2)
3    (600, 6, 2)
4    (599, 6, 2)
Name: agent_locations, dtype: object
agent_nose
0    (600, 2)
1    (600, 2)
2    (600, 2)
3    (600, 2)
4    (599, 2)
Name: agent_nose, dtype: object
agent_nose_to_reward_port
0    (600,)
1    (600,)
2    (600,)
3    (600,)
4    (599,)
Name: agent_nose_to_reward_port, dtype: object
agent_thorax
0    (600, 2)
1    (600, 2)
2    (600, 2)
3    (600, 2)
4    (599, 2)
Name: agent_thorax, dtype: object
agent_thorax_velocity
0    (600,)
1    (600,)
2    (600,)
3    (600,)
4    (599,)
Name: agent_thorax_velocity, dtype: object
closebool_agent_nose_to_reward_port
0    (600,)
1    (600,)
2    (600,)
3    (600,)
4    (599,)
Name: closebool_agent_nose_to_reward_port, dtype: object
closebool_subject_nose_to_reward_port
0    (600,)
1    (600,)
2    (600,)
3    (600,)
4    (599,)
Name: closebool_subject_nose_to_reward_port, dtype: object
clusterable_embedding_x
0    (600,)
1    (600,)
2    (600,)
3   

In [72]:
TRIALS_AND_SPECTRAL_DF["frame_index"]

0      [1086, 1087, 1088, 1089, 1090, 1091, 1092, 109...
1      [1086, 1087, 1088, 1089, 1090, 1091, 1092, 109...
2      [3082, 3083, 3084, 3085, 3086, 3087, 3088, 308...
3      [3082, 3083, 3084, 3085, 3086, 3087, 3088, 308...
4      [4280, 4281, 4282, 4283, 4284, 4285, 4286, 428...
                             ...                        
629    [51709, 51710, 51711, 51712, 51713, 51714, 517...
630    [53904, 53905, 53906, 53907, 53908, 53909, 539...
631    [53904, 53905, 53906, 53907, 53908, 53909, 539...
632    [55003, 55004, 55005, 55006, 55007, 55008, 550...
633    [55003, 55004, 55005, 55006, 55007, 55008, 550...
Name: frame_index, Length: 634, dtype: object

In [73]:
TRIALS_AND_SPECTRAL_DF["cluster_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["video_timestamps"][x["frame_index"]] if isinstance(x["frame_index"], np.ndarray) else np.nan, axis=1)

In [74]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"]["cluster_timestamp"].iloc[0]

array([1087906, 1089292, 1089292, 1090678, 1092063, 1093449, 1093449,
       1094835, 1096221, 1097342, 1097607, 1098993, 1100379, 1101764,
       1101764, 1103150, 1104536, 1104536, 1105922, 1107308, 1107308,
       1108694, 1110080, 1111466, 1111466, 1112851, 1114237, 1114237,
       1115623, 1117009, 1118395, 1118395, 1119781, 1121167, 1122553,
       1122553, 1123938, 1125324, 1126710, 1126710, 1128096, 1129482,
       1129482, 1130868, 1132254, 1133639, 1133639, 1135025, 1136411,
       1136411, 1137797, 1139183, 1140569, 1141295, 1141955, 1143341,
       1143374, 1144726, 1146112, 1147498, 1148884, 1148884, 1150270,
       1151656, 1151656, 1153042, 1154428, 1155813, 1155813, 1157199,
       1158585, 1158585, 1159971, 1161357, 1161357, 1162743, 1164129,
       1165514, 1165514, 1166900, 1168286, 1168839, 1169672, 1171058,
       1172444, 1173830, 1173830, 1175216, 1176601, 1176601, 1177987,
       1179373, 1180759, 1180759, 1182145, 1183531, 1183531, 1184917,
       1186303, 1187

In [75]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"]["cluster_timestamp"].iloc[0].shape

(600,)

In [76]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"]

0       1287955
1       1287955
2       3287980
3       3287980
4       4487995
         ...   
629    52006282
630    54206307
631    54206307
632    55306318
633    55306318
Name: tone_start_timestamp, Length: 634, dtype: int64

In [77]:
sleap_columns

['agent_locations',
 'agent_nose',
 'agent_nose_to_reward_port',
 'agent_thorax',
 'agent_thorax_velocity',
 'closebool_agent_nose_to_reward_port',
 'closebool_subject_nose_to_reward_port',
 'clusterable_embedding_x',
 'clusterable_embedding_y',
 'competitiveness',
 'frame_index',
 'kmeans_cluster',
 'manual_cluster_id',
 'movingbool_agent_thorax_velocity',
 'movingbool_subject_thorax_velocity',
 'nose_to_reward_port_diff',
 'nose_to_reward_port_sum',
 'nose_to_tail_diff',
 'nose_to_tail_sum',
 'standard_embedding_x',
 'standard_embedding_y',
 'subject_locations',
 'subject_nose',
 'subject_nose_to_reward_port',
 'subject_thorax',
 'subject_thorax_to_agent_thorax',
 'subject_thorax_velocity',
 'thorax_velocity_diff',
 'thorax_velocity_sum',
 'within_trial_frame_index']

In [78]:
for col in sorted(sleap_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[1] if isinstance(x["cluster_timestamp"], np.ndarray) else np.nan, axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[0] if isinstance(x["cluster_timestamp"], np.ndarray) else np.nan, axis=1)

trial_agent_locations
trial_agent_nose
trial_agent_nose_to_reward_port
trial_agent_thorax
trial_agent_thorax_velocity
trial_closebool_agent_nose_to_reward_port
trial_closebool_subject_nose_to_reward_port
trial_clusterable_embedding_x
trial_clusterable_embedding_y
trial_competitiveness
trial_frame_index
trial_kmeans_cluster
trial_manual_cluster_id
trial_movingbool_agent_thorax_velocity
trial_movingbool_subject_thorax_velocity
trial_nose_to_reward_port_diff
trial_nose_to_reward_port_sum
trial_nose_to_tail_diff
trial_nose_to_tail_sum
trial_standard_embedding_x
trial_standard_embedding_y
trial_subject_locations
trial_subject_nose
trial_subject_nose_to_reward_port
trial_subject_thorax
trial_subject_thorax_to_agent_thorax
trial_subject_thorax_velocity
trial_thorax_velocity_diff
trial_thorax_velocity_sum
trial_within_trial_frame_index


In [79]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"].head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,trial_subject_locations,trial_subject_nose,trial_subject_nose_to_reward_port,trial_subject_thorax,trial_subject_thorax_to_agent_thorax,trial_subject_thorax_velocity,trial_thorax_velocity_diff,trial_thorax_velocity_sum,trial_within_trial_frame_index,trial_video_timestamps
0,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[[[58.7109255019664, 32.67840814964088], [55.2...","[[59.19453014666447, 35.289833902858106], [59....","[2.3596265, 2.2175457, 2.0829062, 1.9465421, 1...","[[53.977658515105496, 30.701062352171952], [53...","[20.362434, 20.359085, 20.2715, 20.158875, 19....","[3.457623, 3.9866521, 4.4569526, 4.839592, 5.1...","[3.942982, 3.8287802, 3.6698003, 3.501487, 3.3...","[6.131656, 6.1688476, 6.2184086, 6.25792, 6.33...","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[1288857, 1288857, 1290243, 1291629, 1291629, ..."
1,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[[[75.21059817027995, 35.1093171902934], [72.4...","[[74.9336190091533, 37.40341132116207], [75.96...","[14.419802, 14.928705, 15.385147, 15.725277, 1...","[[74.46577580035895, 32.50361589464441], [73.7...","[20.362434, 20.359085, 20.2715, 20.158875, 19....","[2.6740332, 2.1821952, 1.7614558, 1.4183275, 1...","[3.942982, 3.8287802, 3.6698003, 3.501487, 3.3...","[6.131656, 6.1688476, 6.2184086, 6.25792, 6.33...","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[1288857, 1288857, 1290243, 1291629, 1291629, ..."
2,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[[[61.40198893840553, 35.92851286762594], [57....","[[60.39633258033338, 38.228024855461534], [60....","[0.47282448, 0.56911826, 0.6200463, 0.614294, ...","[[61.16088919348937, 33.65302041958023], [61.1...","[17.451311, 17.744617, 18.004671, 18.235722, 1...","[1.2043722, 1.150706, 1.107268, 1.0628508, 1.0...","[15.230153, 13.869027, 12.124398, 10.202402, 8...","[17.554428, 16.05088, 14.199957, 12.173633, 10...","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[3288667, 3288667, 3290053, 3291438, 3292824, ..."
3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[[[43.896036339278204, 19.280670419213397], [4...","[[42.417249101290416, 17.80652164153777], [42....","[25.322828, 25.833288, 26.260775, 26.637802, 2...","[[49.0884531973002, 19.263635390682467], [49.0...","[17.451311, 17.744617, 18.004671, 18.235722, 1...","[16.350056, 14.900174, 13.0926895, 11.110782, ...","[15.230153, 13.869027, 12.124398, 10.202402, 8...","[17.554428, 16.05088, 14.199957, 12.173633, 10...","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[3288667, 3288667, 3290053, 3291438, 3292824, ..."
4,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[[[61.425272749554395, 36.6142101563131], [59....","[[59.891351523460834, 37.729727836741496], [59...","[0.6079607, 0.6064874, 0.60580224, 0.6068849, ...","[[65.31326854476629, 34.47063448649385], [65.3...","[15.918593, 15.208826, 14.559065, 13.927606, 1...","[0.6456263, 0.6806483, 0.7336328, 0.7997021, 0...","[19.38422, 19.77671, 20.036701, 20.128536, 20....","[20.675472, 21.138006, 21.503967, 21.727942, 2...","[200, 201, 202, 203, 204, 205, 206, 207, 208, ...","[4488830, 4488830, 4490216, 4491601, 4492351, ..."


In [80]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"]["trial_frame_index"].iloc[0].shape

(200,)

In [81]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["cluster_timestamp"], errors="ignore")

In [82]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition
competition_closeness
get_reward_frame
out_reward_frame
experiment
video_name
tone_start_timestamp
tone_stop_timestamp
cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
vHPC_lfp_trace
BLA_lfp_trace
LH_lfp_trace
MD_lfp_trace
mPFC_lfp_trace
vHPC_theta_band
BLA_theta_band
LH_theta_band
MD_theta_band
mPFC_theta_band
vHPC_gamma_band
BLA_gamma_band
LH_gamma_band
MD_gamma_band
mPFC_gamma_band
vHPC_theta_phase
BLA_theta_phase
LH_theta_phase
MD_theta_phase
mPFC_theta_phase
vHPC_gamma_phase
BLA_gamma_phase
LH_gamma_phase
MD_gamma_phase
mPFC_gamma_phase
vHPC_power_all_frequencies_all_windows
BLA_power_all_frequencies_all_windows
LH_power_all_frequencies_all_windows
MD_power_all_frequencies_all_windo

- Filtering coherence

In [83]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [84]:
coherence_columns

['BLA_LH_coherence_all_frequencies_all_windows',
 'BLA_MD_coherence_all_frequencies_all_windows',
 'BLA_mPFC_coherence_all_frequencies_all_windows',
 'BLA_vHPC_coherence_all_frequencies_all_windows',
 'LH_MD_coherence_all_frequencies_all_windows',
 'LH_mPFC_coherence_all_frequencies_all_windows',
 'LH_vHPC_coherence_all_frequencies_all_windows',
 'MD_mPFC_coherence_all_frequencies_all_windows',
 'MD_vHPC_coherence_all_frequencies_all_windows',
 'mPFC_vHPC_coherence_all_frequencies_all_windows',
 'BLA_LH_coherence_theta',
 'BLA_LH_coherence_gamma',
 'BLA_MD_coherence_theta',
 'BLA_MD_coherence_gamma',
 'BLA_mPFC_coherence_theta',
 'BLA_mPFC_coherence_gamma',
 'BLA_vHPC_coherence_theta',
 'BLA_vHPC_coherence_gamma',
 'LH_MD_coherence_theta',
 'LH_MD_coherence_gamma',
 'LH_mPFC_coherence_theta',
 'LH_mPFC_coherence_gamma',
 'LH_vHPC_coherence_theta',
 'LH_vHPC_coherence_gamma',
 'MD_mPFC_coherence_theta',
 'MD_mPFC_coherence_gamma',
 'MD_vHPC_coherence_theta',
 'MD_vHPC_coherence_gamma',


In [85]:
for col in sorted(coherence_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_coherence_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

trial_BLA_LH_coherence_all_frequencies_all_windows
trial_BLA_LH_coherence_gamma
trial_BLA_LH_coherence_theta
trial_BLA_MD_coherence_all_frequencies_all_windows
trial_BLA_MD_coherence_gamma
trial_BLA_MD_coherence_theta
trial_BLA_mPFC_coherence_all_frequencies_all_windows
trial_BLA_mPFC_coherence_gamma
trial_BLA_mPFC_coherence_theta
trial_BLA_vHPC_coherence_all_frequencies_all_windows
trial_BLA_vHPC_coherence_gamma
trial_BLA_vHPC_coherence_theta
trial_LH_MD_coherence_all_frequencies_all_windows
trial_LH_MD_coherence_gamma
trial_LH_MD_coherence_theta
trial_LH_mPFC_coherence_all_frequencies_all_windows
trial_LH_mPFC_coherence_gamma
trial_LH_mPFC_coherence_theta
trial_LH_vHPC_coherence_all_frequencies_all_windows
trial_LH_vHPC_coherence_gamma
trial_LH_vHPC_coherence_theta
trial_MD_mPFC_coherence_all_frequencies_all_windows
trial_MD_mPFC_coherence_gamma
trial_MD_mPFC_coherence_theta
trial_MD_vHPC_coherence_all_frequencies_all_windows
trial_MD_vHPC_coherence_gamma
trial_MD_vHPC_coherence_thet

In [86]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [87]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [88]:
granger_columns

['BLA_LH_granger_all_frequencies_all_windows',
 'LH_BLA_granger_all_frequencies_all_windows',
 'BLA_MD_granger_all_frequencies_all_windows',
 'MD_BLA_granger_all_frequencies_all_windows',
 'BLA_mPFC_granger_all_frequencies_all_windows',
 'mPFC_BLA_granger_all_frequencies_all_windows',
 'BLA_vHPC_granger_all_frequencies_all_windows',
 'vHPC_BLA_granger_all_frequencies_all_windows',
 'LH_MD_granger_all_frequencies_all_windows',
 'MD_LH_granger_all_frequencies_all_windows',
 'LH_mPFC_granger_all_frequencies_all_windows',
 'mPFC_LH_granger_all_frequencies_all_windows',
 'LH_vHPC_granger_all_frequencies_all_windows',
 'vHPC_LH_granger_all_frequencies_all_windows',
 'MD_mPFC_granger_all_frequencies_all_windows',
 'mPFC_MD_granger_all_frequencies_all_windows',
 'MD_vHPC_granger_all_frequencies_all_windows',
 'vHPC_MD_granger_all_frequencies_all_windows',
 'mPFC_vHPC_granger_all_frequencies_all_windows',
 'vHPC_mPFC_granger_all_frequencies_all_windows',
 'BLA-to-LH_granger_theta',
 'BLA-to-LH_

In [89]:
for col in sorted(granger_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_granger_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

trial_BLA-to-LH_granger_gamma
trial_BLA-to-LH_granger_theta
trial_BLA-to-MD_granger_gamma
trial_BLA-to-MD_granger_theta
trial_BLA-to-mPFC_granger_gamma
trial_BLA-to-mPFC_granger_theta
trial_BLA-to-vHPC_granger_gamma
trial_BLA-to-vHPC_granger_theta
trial_BLA_LH_granger_all_frequencies_all_windows
trial_BLA_MD_granger_all_frequencies_all_windows
trial_BLA_mPFC_granger_all_frequencies_all_windows
trial_BLA_vHPC_granger_all_frequencies_all_windows
trial_LH-to-BLA_granger_gamma
trial_LH-to-BLA_granger_theta
trial_LH-to-MD_granger_gamma
trial_LH-to-MD_granger_theta
trial_LH-to-mPFC_granger_gamma
trial_LH-to-mPFC_granger_theta
trial_LH-to-vHPC_granger_gamma
trial_LH-to-vHPC_granger_theta
trial_LH_BLA_granger_all_frequencies_all_windows
trial_LH_MD_granger_all_frequencies_all_windows
trial_LH_mPFC_granger_all_frequencies_all_windows
trial_LH_vHPC_granger_all_frequencies_all_windows
trial_MD-to-BLA_granger_gamma
trial_MD-to-BLA_granger_theta
trial_MD-to-LH_granger_gamma
trial_MD-to-LH_granger_t

In [90]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

In [91]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

- Filtering power

In [92]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,trial_vHPC-to-LH_granger_theta,trial_vHPC-to-MD_granger_gamma,trial_vHPC-to-MD_granger_theta,trial_vHPC-to-mPFC_granger_gamma,trial_vHPC-to-mPFC_granger_theta,trial_vHPC_BLA_granger_all_frequencies_all_windows,trial_vHPC_LH_granger_all_frequencies_all_windows,trial_vHPC_MD_granger_all_frequencies_all_windows,trial_vHPC_mPFC_granger_all_frequencies_all_windows,trial_granger_timestamps
0,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.9009608, 0.50006056, 0.40058264, 0.20688353...","[0.25817865, 0.30094522, 0.699677, 0.40107265,...","[0.561425, 0.5851245, 0.65318984, 0.3532337, 0...","[0.32194954, 0.40533137, 0.18068044, 0.2594404...","[0.7571986, 0.815081, 0.25992066, 0.26617825, ...","[[1.9081154, 0.60081196, 1.1654721, 1.9988519,...","[[0.9983736, 0.47073475, 0.343659, 1.7212274, ...","[[0.17532644, 0.058493987, 0.21699055, 0.98471...","[[nan, 0.020071205, 0.055330258, 0.6095272, 0....","[1290000, 1300000, 1310000, 1320000, 1330000, ..."
1,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.18645912, 0.08189094, 0.2744491, 0.23336548...","[0.47432554, 0.4996, 0.297654, 0.33931163, 0.3...","[0.28323543, 0.118440665, 0.40134224, 0.176665...","[0.21483609, 0.29609802, 0.2730743, 0.32005793...","[0.40129688, 0.12351816, 0.1833222, 0.26456228...","[[0.9112918, 0.4171988, 0.8085533, 0.2920456, ...","[[0.40942863, 0.046180055, 0.19375496, 0.01620...","[[0.60639507, 0.31954294, 0.724143, 0.43387735...","[[3.6664703, 0.42619234, 1.4205563, 0.29950142...","[1290000, 1300000, 1310000, 1320000, 1330000, ..."
2,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.4083339, nan, nan, 0.40544108, 0.713948, na...","[0.37535805, 0.6207084, 0.51570064, 0.48562723...","[1.0129654, 1.9590354, 0.81798965, 0.44662973,...","[0.34424874, 0.49313703, 0.62455237, 0.7944606...","[0.27219552, 0.4817125, 0.33464476, 0.47396374...","[[0.15322271, 0.46380085, 0.62996185, 3.110912...","[[0.24907123, 0.016390795, 0.29728585, 0.86932...","[[0.010317478, 0.031716213, 0.27721742, 0.7599...","[[0.00029082445, 0.042989183, 0.7334191, 0.394...","[3290000, 3300000, 3310000, 3320000, 3330000, ..."
3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.3853952, 0.25268877, nan, nan, 0.31868017, ...","[0.2541444, 0.20815888, nan, nan, 0.19152872, ...","[0.36448398, 0.9780728, nan, nan, 0.61114615, ...","[0.2837622, 0.31374303, nan, nan, 0.3473629, n...","[0.4404981, 0.56984967, nan, nan, 0.3923128, n...","[[nan, 1.1006346, 1.7227364, 0.62379414, 1.976...","[[nan, 1.5132141, 1.5424117, 0.9611375, 1.6241...","[[1.6970849, 1.3136865, 1.0092934, 1.0669379, ...","[[1.2902783, 1.146602, 0.89429384, 0.86628497,...","[3290000, 3300000, 3310000, 3320000, 3330000, ..."
4,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.2865354, 0.19214727, 0.37286362, 0.96648353...","[nan, nan, 0.4565851, 0.44872823, 0.1648814, 0...","[nan, nan, 0.14125006, 0.72149754, 0.1910801, ...","[0.12474145, 0.36122033, 0.40298712, 0.4058549...","[0.3782114, 0.28635412, 0.110780016, 0.4070134...","[[0.405383, 1.4240549, 1.2131125, 0.4940806, 1...","[[0.28156158, 1.0402817, nan, 0.85418475, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[0.055893026, 0.17280938, 1.5078031, 0.661239...","[4490000, 4500000, 4510000, 4520000, 4530000, ..."


In [93]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [94]:
power_columns

['vHPC_power_all_frequencies_all_windows',
 'BLA_power_all_frequencies_all_windows',
 'LH_power_all_frequencies_all_windows',
 'MD_power_all_frequencies_all_windows',
 'mPFC_power_all_frequencies_all_windows',
 'vHPC_power_theta',
 'vHPC_power_gamma',
 'BLA_power_theta',
 'BLA_power_gamma',
 'LH_power_theta',
 'LH_power_gamma',
 'MD_power_theta',
 'MD_power_gamma',
 'mPFC_power_theta',
 'mPFC_power_gamma']

In [95]:
for col in sorted(power_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_power_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)


trial_BLA_power_all_frequencies_all_windows
trial_BLA_power_gamma
trial_BLA_power_theta
trial_LH_power_all_frequencies_all_windows
trial_LH_power_gamma
trial_LH_power_theta
trial_MD_power_all_frequencies_all_windows
trial_MD_power_gamma
trial_MD_power_theta
trial_mPFC_power_all_frequencies_all_windows
trial_mPFC_power_gamma
trial_mPFC_power_theta
trial_vHPC_power_all_frequencies_all_windows
trial_vHPC_power_gamma
trial_vHPC_power_theta


In [96]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

In [97]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition
competition_closeness
get_reward_frame
out_reward_frame
experiment
video_name
tone_start_timestamp
tone_stop_timestamp
cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
vHPC_lfp_trace
BLA_lfp_trace
LH_lfp_trace
MD_lfp_trace
mPFC_lfp_trace
vHPC_theta_band
BLA_theta_band
LH_theta_band
MD_theta_band
mPFC_theta_band
vHPC_gamma_band
BLA_gamma_band
LH_gamma_band
MD_gamma_band
mPFC_gamma_band
vHPC_theta_phase
BLA_theta_phase
LH_theta_phase
MD_theta_phase
mPFC_theta_phase
vHPC_gamma_phase
BLA_gamma_phase
LH_gamma_phase
MD_gamma_phase
mPFC_gamma_phase
power_calculation_frequencies
coherence_calculation_frequencies
granger_calculation_frequencies
tone_start_frame_remove
agent_tail_base
agent_to_rew

# Filtering out phase

In [98]:
lfp_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col and "timestamps" not in col and "calculation" not in col]

In [99]:
band_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "band" in col and "timestamps" not in col and "calculation" not in col]

In [100]:
phase_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "phase" in col and "timestamps" not in col and "calculation" not in col]

In [101]:
phase_columns = phase_columns + band_columns +lfp_columns

In [102]:
phase_columns

['vHPC_theta_phase',
 'BLA_theta_phase',
 'LH_theta_phase',
 'MD_theta_phase',
 'mPFC_theta_phase',
 'vHPC_gamma_phase',
 'BLA_gamma_phase',
 'LH_gamma_phase',
 'MD_gamma_phase',
 'mPFC_gamma_phase',
 'vHPC_theta_band',
 'BLA_theta_band',
 'LH_theta_band',
 'MD_theta_band',
 'mPFC_theta_band',
 'vHPC_gamma_band',
 'BLA_gamma_band',
 'LH_gamma_band',
 'MD_gamma_band',
 'mPFC_gamma_band',
 'vHPC_lfp_trace',
 'BLA_lfp_trace',
 'LH_lfp_trace',
 'MD_lfp_trace',
 'mPFC_lfp_trace']

In [103]:
for col in sorted(phase_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_lfp_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)


trial_BLA_gamma_band
trial_BLA_gamma_phase
trial_BLA_lfp_trace
trial_BLA_theta_band
trial_BLA_theta_phase
trial_LH_gamma_band
trial_LH_gamma_phase
trial_LH_lfp_trace
trial_LH_theta_band
trial_LH_theta_phase
trial_MD_gamma_band
trial_MD_gamma_phase
trial_MD_lfp_trace
trial_MD_theta_band
trial_MD_theta_phase
trial_mPFC_gamma_band
trial_mPFC_gamma_phase
trial_mPFC_lfp_trace
trial_mPFC_theta_band
trial_mPFC_theta_phase
trial_vHPC_gamma_band
trial_vHPC_gamma_phase
trial_vHPC_lfp_trace
trial_vHPC_theta_band
trial_vHPC_theta_phase


In [104]:
TRIALS_AND_SPECTRAL_DF["trial_lfp_timestamps"]

0      [1287960, 1287980, 1288000, 1288020, 1288040, ...
1      [1287960, 1287980, 1288000, 1288020, 1288040, ...
2      [3287980, 3288000, 3288020, 3288040, 3288060, ...
3      [3287980, 3288000, 3288020, 3288040, 3288060, ...
4      [4488000, 4488020, 4488040, 4488060, 4488080, ...
                             ...                        
629    [52006300, 52006320, 52006340, 52006360, 52006...
630    [54206320, 54206340, 54206360, 54206380, 54206...
631    [54206320, 54206340, 54206360, 54206380, 54206...
632    [55306320, 55306340, 55306360, 55306380, 55306...
633    [55306320, 55306340, 55306360, 55306380, 55306...
Name: trial_lfp_timestamps, Length: 634, dtype: object

In [105]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=phase_columns + ["lfp_timestamps"], errors="ignore")

In [106]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [107]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition
competition_closeness
get_reward_frame
out_reward_frame
experiment
video_name
tone_start_timestamp
tone_stop_timestamp
cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
power_calculation_frequencies
coherence_calculation_frequencies
granger_calculation_frequencies
tone_start_frame_remove
agent_tail_base
agent_to_reward_port_angle
subject_tail_base
subject_to_reward_port_angle
to_reward_port_angle_diff
to_reward_port_angle_sum
within_trial_index
agent
10s_after_tone_frame
10s_before_tone_frame
all_subjects_remove
body_parts
box_bottom_left
box_bottom_right
box_number
box_top_left
box_top_right
color
comp_id
competition_closeness_remove
condition 
experiment_remove
first_timestamp_remove
get_reward_frame_

In [108]:
TRIALS_AND_SPECTRAL_DF["trial_kmeans_cluster"].iloc[0].shape

(200,)

In [109]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [110]:
raise ValueError()

ValueError: 

# Filtering for rows that are in the video

In [ ]:
TRIALS_AND_SPECTRAL_DF["in_video"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["start_frame"] <= x["trial_video_frame"][0] <= x["stop_frame"], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["in_video"]].reset_index()

In [ ]:
TRIALS_AND_SPECTRAL_DF

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError()

# Filtering out spikes

In [ ]:
raise ValueError()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1], axis=1)
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1].T, axis=1)
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=["spike_clusters", "spike_times", "neuron_average_fr", "neuron_average_timestamps",], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"].iloc[0].shape